In [1]:
# Import packages
import csv
from flask import Flask, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify
import io
import json
from json import dumps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from psycopg2 import connect
from sqlalchemy import create_engine
import sqlite3

%matplotlib inline

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Check out historical coin data
df_coin = pd.read_csv('historical_coin_data.csv')
df_coin.timestamp = pd.to_datetime(df_coin.timestamp)
df_coin.set_index('timestamp', inplace=True)
df_coin.head()

coins        price  sequence       size  \
timestamp                                                            
2018-07-16 20:11:51.747745   ETH   427.921875      4068  66.599176   
2018-07-16 20:11:52.628113   BTC  6101.377288      4081  96.144496   
2018-07-16 20:11:53.164516   BTC  5842.166335      4134  25.551513   
2018-07-16 20:11:53.722737   ETH   437.650186      4205   2.932149   
2018-07-16 20:11:54.050962   BTC  5453.596138      4237  72.177044   

                                        symbol_id      taker_side  \
timestamp                                                           
2018-07-16 20:11:51.747745  COINBASE_SPOT_ETH_USD  SELL_ESTIMATED   
2018-07-16 20:11:52.628113  COINBASE_SPOT_BTC_USD  SELL_ESTIMATED   
2018-07-16 20:11:53.164516  COINBASE_SPOT_BTC_USD  SELL_ESTIMATED   
2018-07-16 20:11:53.722737  COINBASE_SPOT_ETH_USD             BUY   
2018-07-16 20:11:54.050962  COINBASE_SPOT_BTC_USD             BUY   

                                          time_exchange   type  \
timestamp                                                        
2018-07-16 20:11:51.747745  2018-07-16T20:11:51.747745Z  trade   
2018-07-16 20:11:52.628113  2018-07-16T20:11:52.628113Z  trade   
2018-07-16 20:11:53.164516  2018-07-16T20:11:53.164516Z  trade   
2018-07-16 20:11:53.722737  2018-07-16T20:11:53.722737Z  trade   
2018-07-16 20:11:54.050962  2018-07-16T20:11:54.050962Z  trade   

                                                            uuid  
timestamp                                                         
2018-07-16 20:11:51.747745  77ba9eed-c909-4903-8315-1bfb96c49b6e  
2018-07-16 20:11:52.628113  0acb98ce-399e-4c7f-bcee-4f86d6eef95c  
2018-07-16 20:11:53.164516  7eaf3d0b-1704-4f4e-8840-24bd64c02dce  
2018-07-16 20:11:53.722737  7f798136-daf7-4b64-af12-f4ecd11006e5  
2018-07-16 20:11:54.050962  5759be1c-d631-463f-8396-06f1239fcbba

In [3]:
# Get timestamp and price data
df_coin = df_coin[['price']]
df_coin.head()

price
timestamp                              
2018-07-16 20:11:51.747745   427.921875
2018-07-16 20:11:52.628113  6101.377288
2018-07-16 20:11:53.164516  5842.166335
2018-07-16 20:11:53.722737   437.650186
2018-07-16 20:11:54.050962  5453.596138

In [4]:
# Plot coin data
df_coin.plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.xlabel('Time', fontsize=20);

In [5]:
# Check out historical sentiment data
df_sent = pd.read_csv('historical_sentiment_data.csv')
df_sent.timestamp = pd.to_datetime(df_sent.timestamp)
df_sent.set_index('timestamp', inplace=True)
df_sent.head()

coins  sentiment_polarity  text
timestamp                                                          
2018-07-16 20:11:50.689683  LTC; BTC; ETH           -0.297049  TEST
2018-07-16 20:11:52.712470  ETH; BTC; LTC            0.244146  TEST
2018-07-16 20:11:54.890689  ETH; LTC; BTC            0.288919  TEST
2018-07-16 20:11:56.363939            BTC            0.315483  TEST
2018-07-16 20:11:57.656349       ETH; BTC           -0.282642  TEST

In [6]:
# Plot sentiment data
df_sent.plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.xlabel('Time', fontsize=20);

In [7]:
# Get timestamp and polarity data
df_sent = df_sent[['sentiment_polarity']]
df_sent.head()

sentiment_polarity
timestamp                                     
2018-07-16 20:11:50.689683           -0.297049
2018-07-16 20:11:52.712470            0.244146
2018-07-16 20:11:54.890689            0.288919
2018-07-16 20:11:56.363939            0.315483
2018-07-16 20:11:57.656349           -0.282642

In [8]:
# Merge data on closest timestamp to compare price versus sentiment polarity
merged = pd.merge_asof(df_coin, df_sent, left_index=True, right_index=True, direction='nearest')
merged

price  sentiment_polarity
timestamp                                                  
2018-07-16 20:11:51.747745   427.921875            0.244146
2018-07-16 20:11:52.628113  6101.377288            0.244146
2018-07-16 20:11:53.164516  5842.166335            0.244146
2018-07-16 20:11:53.722737   437.650186            0.244146
2018-07-16 20:11:54.050962  5453.596138            0.288919
2018-07-16 20:11:54.572711   476.548393            0.288919
2018-07-16 20:11:55.110619    77.700954            0.288919
2018-07-16 20:11:55.728178   492.019250            0.315483
2018-07-16 20:11:56.522702  5374.669683            0.315483
2018-07-16 20:11:57.278040   513.920570           -0.282642
2018-07-16 20:11:58.038304    76.339935           -0.282642
2018-07-16 20:11:58.557515   496.205408           -0.282642
2018-07-16 20:11:59.405121   477.335011            0.247427
2018-07-16 20:11:59.842484   475.964245            0.247427
2018-07-16 20:12:00.602693   482.461917            0.247427
2018-07-16 20:12:01.328111    79.141019           -0.914358
2018-07-16 20:12:02.010997   512.114265           -0.914358
2018-07-16 20:12:02.467279    78.692978           -0.914358
2018-07-16 20:12:03.246813    81.995498            0.677265
2018-07-16 20:12:03.816497  5491.265823            0.677265
2018-07-16 20:12:04.594858    87.661916           -0.967297
2018-07-16 20:12:05.322644    91.517813           -0.967297
2018-07-16 20:12:06.138033  5343.920700           -0.967297
2018-07-16 20:12:06.849516    87.137464           -0.967297
2018-07-16 20:12:07.586091    88.107714            0.502659
2018-07-16 20:12:08.093417  5038.992805            0.502659
2018-07-16 20:12:08.795957    88.699100            0.502659
2018-07-16 20:12:09.633109   512.563413           -0.471525
2018-07-16 20:12:10.195509    85.081987           -0.471525
2018-07-16 20:12:10.979710    86.655164           -0.471525
...                                 ...                 ...
2018-07-16 20:14:41.785948  4064.320035           -0.081332
2018-07-16 20:14:42.275592    47.325382            0.839542
2018-07-16 20:14:42.940745    47.171114            0.839542
2018-07-16 20:14:43.262258    49.018102            0.839542
2018-07-16 20:14:43.629752   511.292907            0.839542
2018-07-16 20:14:44.284870  3844.421640           -0.142412
2018-07-16 20:14:44.662709    51.220830           -0.142412
2018-07-16 20:14:45.367201    51.541790           -0.142412
2018-07-16 20:14:45.906918  4101.050774           -0.142412
2018-07-16 20:14:46.487714    52.041986            0.212892
2018-07-16 20:14:47.311105    53.958489            0.212892
2018-07-16 20:14:48.068561   528.470930            0.212892
2018-07-16 20:14:48.815050   559.112034            0.196172
2018-07-16 20:14:49.327533  4132.902919            0.196172
2018-07-16 20:14:49.663833    50.214192            0.196172
2018-07-16 20:14:50.469352  4216.298648            0.196172
2018-07-16 20:14:51.060835    51.061605            0.196172
2018-07-16 20:14:51.788224    52.122146            0.247862
2018-07-16 20:14:52.685542    54.040836            0.247862
2018-07-16 20:14:53.413688  4353.795710            0.247862
2018-07-16 20:14:54.035020  4521.395948           -0.731381
2018-07-16 20:14:54.563121   476.794745           -0.731381
2018-07-16 20:14:54.987980  4399.315818           -0.731381
2018-07-16 20:14:55.689707    54.087054           -0.731381
2018-07-16 20:14:56.097441   475.174224           -0.317282
2018-07-16 20:14:56.685897  4343.618138           -0.317282
2018-07-16 20:14:57.062496   508.252609           -0.317282
2018-07-16 20:14:57.478370  4372.094097           -0.317282
2018-07-16 20:14:58.002077  4058.744282           -0.895725
2018-07-16 20:14:58.357503  4269.915336           -0.895725

[308 rows x 2 columns]

In [9]:
# Preview correlation of merged data
merged.corr()

price  sentiment_polarity
price               1.000000            0.106937
sentiment_polarity  0.106937            1.000000

In [10]:
# Turn correlation data into simpler format for database load
def correlation():
    first = merged.index[0]
    print("First Timestamp: ", first)
    
    last = merged.index[-1]
    print("Last Timestamp: ", last)
    
    corr = merged.corr().iloc[1, 0]
    print("Correlation Coefficient: ", corr)
    
    df = pd.DataFrame()
    df['first timestamp'] = [first]
    df['last timestamp'] = [last]
    df['correlation'] = [corr]

    return df

In [11]:
# Load correlation and merged data sets into sqlite3
def load_correlation():
    data = correlation()
    engine = create_engine('sqlite:///cryptomood.db', echo=False)
    data.to_sql(name='correlation', con=engine, if_exists = 'append', index=False)
    merged.to_sql(name='merged_data', con=engine, if_exists = 'append', index=False)

load_correlation()
print("\nData successfully loaded into sqlite3")

First Timestamp:  2018-07-16 20:11:51.747745
Last Timestamp:  2018-07-16 20:14:58.357503
Correlation Coefficient:  0.10693711137928621

Data successfully loaded into sqlite3


In [12]:
# Upload historical coin data and preview data
def load_historical():
    db_connect = sqlite3.connect('cryptomood.db')
    cur = db_connect.cursor()

    csv = "historical_coin_data.csv"
    df = pd.read_csv(csv)
    df.to_sql('historical_coin_data', db_connect, if_exists='append', index=False)

    query =  cur.execute("SELECT * FROM historical_coin_data LIMIT 5;")
    fetch = cur.fetchall()
    
    columns = [d[0] for d in cur.description]
    result = [dict(zip(columns, row)) for row in fetch]
    
    return json.dumps(result)

results = load_historical()
parsed = json.loads(results)
print(json.dumps(parsed, indent=4, sort_keys=True))

[
    {
        "coins": "ETH",
        "price": 427.92187479999996,
        "sequence": 4068,
        "size": 66.59917596,
        "symbol_id": "COINBASE_SPOT_ETH_USD",
        "taker_side": "SELL_ESTIMATED",
        "time_exchange": "2018-07-16T20:11:51.747745Z",
        "timestamp": "2018-07-16T20:11:51.747745Z",
        "type": "trade",
        "uuid": "77ba9eed-c909-4903-8315-1bfb96c49b6e"
    },
    {
        "coins": "BTC",
        "price": 6101.377288000001,
        "sequence": 4081,
        "size": 96.14449585,
        "symbol_id": "COINBASE_SPOT_BTC_USD",
        "taker_side": "SELL_ESTIMATED",
        "time_exchange": "2018-07-16T20:11:52.628113Z",
        "timestamp": "2018-07-16T20:11:52.628113Z",
        "type": "trade",
        "uuid": "0acb98ce-399e-4c7f-bcee-4f86d6eef95c"
    },
    {
        "coins": "BTC",
        "price": 5842.166335,
        "sequence": 4134,
        "size": 25.5515129,
        "symbol_id": "COINBASE_SPOT_BTC_USD",
        "taker_side": "SELL_ESTIM

In [13]:
# Upload historical sentiment data and preview data
def load_sentiment():
    db_connect = sqlite3.connect('cryptomood.db')
    cur = db_connect.cursor()
    
    csv = "historical_sentiment_data.csv"
    df = pd.read_csv(csv)
    df.to_sql('historical_sentiment_data', db_connect, if_exists='append', index=False)

    cur.execute("SELECT * FROM historical_sentiment_data LIMIT 5;")
    fetch = cur.fetchall()
    
    columns = [d[0] for d in cur.description]
    result = [dict(zip(columns, row)) for row in fetch]
    
    return json.dumps(result)

results = load_sentiment()
parsed = json.loads(results)
print(json.dumps(parsed, indent=4, sort_keys=True))

[
    {
        "coins": "LTC; BTC; ETH",
        "sentiment_polarity": -0.297048641,
        "text": "TEST",
        "timestamp": "2018-07-16T20:11:50.689683Z"
    },
    {
        "coins": "ETH; BTC; LTC",
        "sentiment_polarity": 0.244145544,
        "text": "TEST",
        "timestamp": "2018-07-16T20:11:52.712470Z"
    },
    {
        "coins": "ETH; LTC; BTC",
        "sentiment_polarity": 0.288919402,
        "text": "TEST",
        "timestamp": "2018-07-16T20:11:54.890689Z"
    },
    {
        "coins": "BTC",
        "sentiment_polarity": 0.315482698,
        "text": "TEST",
        "timestamp": "2018-07-16T20:11:56.363939Z"
    },
    {
        "coins": "ETH; BTC",
        "sentiment_polarity": -0.282641706,
        "text": "TEST",
        "timestamp": "2018-07-16T20:11:57.656349Z"
    }
]


In [ ]:
# REST APIs serving the data for consumption
db_connect = create_engine('sqlite:///cryptomood.db')
app = Flask(__name__)
api = Api(app)

class Coin(Resource):
    def get(self):
        conn = db_connect.connect()
        query = conn.execute("SELECT * FROM historical_coin_data;")
        result = {'data': [dict(zip(tuple (query.keys()), i)) for i in query.cursor]}
        return jsonify(result)
    
class Sentiment(Resource):
    def get(self):
        conn = db_connect.connect()
        query = conn.execute("SELECT * FROM historical_sentiment_data;")
        result = {'data': [dict(zip(tuple (query.keys()), i)) for i in query.cursor]}
        return jsonify(result)

class Correlation(Resource):
    def get(self):
        conn = db_connect.connect()
        query = conn.execute("SELECT * FROM correlation;")
        result = {'data': [dict(zip(tuple (query.keys()), i)) for i in query.cursor]}
        return jsonify(result)

class Merged(Resource):
    def get(self):
        conn = db_connect.connect()
        query = conn.execute("SELECT * FROM merged_data;")
        result = {'data': [dict(zip(tuple (query.keys()), i)) for i in query.cursor]}
        return jsonify(result)
    
api.add_resource(Coin, '/coin') # Route 1
api.add_resource(Sentiment, '/sentiment') # Route 2
api.add_resource(Correlation, '/correlation') # Route 3
api.add_resource(Merged, '/merged') # Route 3

if __name__ == '__main__':
     app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2019 10:34:54] "GET /correlation HTTP/1.1" 200 -
